In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
XAUUSD Alert Bot (SQLite -> 1m candles -> signals -> alert only when advice)
- Auto-detects tick table & columns in SQLite (ts + price)
- Builds 1-minute candles from ticks
- Computes: EMA(20/50), Bollinger(20,2), MACD(12,26,9), ATR(14)
- Signals:
  A) Pullback Continuation LONG (trend + pullback + bullish trigger)
  B) Break & Hold LONG (break above recent range + hold confirmation)
  C) Failure SHORT (breakdown + retest rejection)  [strict, optional]
- Alerts:
  - Console always
  - Optional Telegram (set env vars: TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID)
- No signal => no message
"""

from __future__ import annotations

import argparse
import os
import sys
import time
import json
import sqlite3
import math
from dataclasses import dataclass
from typing import Optional, Tuple, Dict, Any, List

import pandas as pd
import numpy as np
import urllib.request


# -----------------------------
# Helpers: SQLite detection
# -----------------------------

TS_CANDIDATES = ["ts", "timestamp", "time", "datetime", "t"]
PRICE_CANDIDATES = ["price", "last", "close", "bid", "ask", "mid", "value"]

def _list_tables(conn: sqlite3.Connection) -> List[str]:
    cur = conn.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%'")
    return [r[0] for r in cur.fetchall()]

def _table_columns(conn: sqlite3.Connection, table: str) -> List[str]:
    cur = conn.execute(f"PRAGMA table_info({table})")
    return [r[1] for r in cur.fetchall()]

def detect_tick_table(conn: sqlite3.Connection,
                      table_hint: Optional[str] = None,
                      ts_hint: Optional[str] = None,
                      price_hint: Optional[str] = None) -> Tuple[str, str, str]:
    tables = _list_tables(conn)
    if not tables:
        raise RuntimeError("Geen tabellen in SQLite database gevonden.")

    if table_hint:
        if table_hint not in tables:
            raise RuntimeError(f"Table '{table_hint}' niet gevonden. Beschikbaar: {tables}")
        tables = [table_hint]

    best = None

    for t in tables:
        cols = _table_columns(conn, t)
        cols_lower = [c.lower() for c in cols]

        ts_col = None
        price_col = None

        if ts_hint and ts_hint in cols:
            ts_col = ts_hint
        else:
            for cand in TS_CANDIDATES:
                if cand in cols_lower:
                    ts_col = cols[cols_lower.index(cand)]
                    break

        if price_hint and price_hint in cols:
            price_col = price_hint
        else:
            for cand in PRICE_CANDIDATES:
                if cand in cols_lower:
                    price_col = cols[cols_lower.index(cand)]
                    break

        if ts_col and price_col:
            # basic validation: can we read something?
            try:
                cur = conn.execute(f"SELECT {ts_col}, {price_col} FROM {t} ORDER BY {ts_col} DESC LIMIT 5")
                rows = cur.fetchall()
                if rows:
                    best = (t, ts_col, price_col)
                    break
            except Exception:
                continue

    if not best:
        raise RuntimeError(
            "Kon geen tick-tabel/kolommen detecteren. "
            "Zorg dat je SQLite een tick-tabel heeft met timestamp (ts/timestamp/...) en price (price/last/...). "
            "Tip: gebruik --table, --ts_col en --price_col om het te forceren."
        )

    return best


# -----------------------------
# Market indicators
# -----------------------------

def ema(series: pd.Series, span: int) -> pd.Series:
    return series.ewm(span=span, adjust=False).mean()

def bollinger(close: pd.Series, length: int = 20, stdev: float = 2.0) -> Tuple[pd.Series, pd.Series, pd.Series]:
    mid = close.rolling(length).mean()
    sd = close.rolling(length).std(ddof=0)
    upper = mid + stdev * sd
    lower = mid - stdev * sd
    return mid, upper, lower

def macd(close: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9) -> Tuple[pd.Series, pd.Series, pd.Series]:
    fast_ema = ema(close, fast)
    slow_ema = ema(close, slow)
    line = fast_ema - slow_ema
    sig = ema(line, signal)
    hist = line - sig
    return line, sig, hist

def atr(df: pd.DataFrame, length: int = 14) -> pd.Series:
    high = df["high"]
    low = df["low"]
    close = df["close"]
    prev_close = close.shift(1)
    tr = pd.concat([
        (high - low),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)
    return tr.rolling(length).mean()


# -----------------------------
# Signal logic
# -----------------------------

@dataclass
class Signal:
    side: str            # "LONG" / "SHORT"
    strategy: str
    entry: float
    sl: float
    tp1: float
    tp2: Optional[float]
    reason: str
    ts_utc: str          # candle time
    confidence: str      # "A" / "B" etc.

def _round(x: float, digits: int = 2) -> float:
    if x is None or (isinstance(x, float) and (math.isnan(x) or math.isinf(x))):
        return float("nan")
    return float(round(x, digits))

def compute_signals(c: pd.DataFrame) -> Optional[Signal]:
    """
    c: candle dataframe indexed by datetime (UTC or naive), columns: open, high, low, close, volume(optional)
    Returns Signal or None
    """

    if len(c) < 120:
        return None

    df = c.copy()
    df["ema20"] = ema(df["close"], 20)
    df["ema50"] = ema(df["close"], 50)
    df["bb_mid"], df["bb_up"], df["bb_low"] = bollinger(df["close"], 20, 2.0)
    df["macd"], df["macd_sig"], df["macd_hist"] = macd(df["close"], 12, 26, 9)
    df["atr14"] = atr(df, 14)

    last = df.iloc[-1]
    prev = df.iloc[-2]
    prev2 = df.iloc[-3]

    # Trend filter
    ema50_slope = df["ema50"].iloc[-1] - df["ema50"].iloc[-6]
    trend_up = (last["close"] > last["ema50"]) and (ema50_slope > 0)
    trend_down = (last["close"] < last["ema50"]) and (ema50_slope < 0)

    # Range levels (recent 60 mins ~ 60 candles)
    lookback = 60
    recent = df.iloc[-lookback:]
    range_high = recent["high"].max()
    range_low = recent["low"].min()

    # -------------------------
    # A) Pullback Continuation LONG (A-setup)
    # Conditions:
    # - trend_up
    # - pullback touched ema20 or bb_mid (in last 5 candles)
    # - bullish trigger: last close > prev high AND last close > ema20
    # - macd_hist improving (rising)
    # -------------------------
    touched_zone = (df["low"].iloc[-5:] <= df["ema20"].iloc[-5:]) | (df["low"].iloc[-5:] <= df["bb_mid"].iloc[-5:])
    bullish_trigger = (last["close"] > prev["high"]) and (last["close"] > last["ema20"])
    macd_improving = (last["macd_hist"] > prev["macd_hist"]) and (prev["macd_hist"] > prev2["macd_hist"])

    if trend_up and touched_zone.any() and bullish_trigger and macd_improving:
        entry = float(last["close"])
        # SL under pullback low minus small buffer
        pullback_low = float(df["low"].iloc[-6:].min())
        buffer = float(last["atr14"] * 0.25) if not np.isnan(last["atr14"]) else 0.5
        sl = pullback_low - buffer
        r = entry - sl
        if r <= 0:
            return None
        tp1 = entry + 1.5 * r
        tp2 = entry + 2.5 * r
        return Signal(
            side="LONG",
            strategy="Pullback Continuation",
            entry=_round(entry),
            sl=_round(sl),
            tp1=_round(tp1),
            tp2=_round(tp2),
            reason="Trend up (EMA50↑), pullback into EMA20/BB-mid, bullish close > prev high, MACD hist improving.",
            ts_utc=str(df.index[-1]),
            confidence="A"
        )

    # -------------------------
    # B) Break & Hold LONG (B+ setup)
    # Conditions:
    # - break above range_high (previous range) by close
    # - and hold: prev close > range_high and last low >= range_high (retest holds)
    # - macd_hist positive or rising
    # -------------------------
    broke = (prev["close"] > range_high) and (prev2["close"] <= range_high)
    held = (last["low"] >= range_high) and (last["close"] >= range_high)
    macd_ok = (last["macd_hist"] >= 0) or (last["macd_hist"] > prev["macd_hist"])

    if broke and held and macd_ok:
        entry = float(last["close"])
        sl = float(range_high - (last["atr14"] * 0.8 if not np.isnan(last["atr14"]) else 1.0))
        r = entry - sl
        if r <= 0:
            return None
        tp1 = entry + 1.5 * r
        tp2 = entry + 2.0 * r
        return Signal(
            side="LONG",
            strategy="Break & Hold",
            entry=_round(entry),
            sl=_round(sl),
            tp1=_round(tp1),
            tp2=_round(tp2),
            reason="Range breakout confirmed: close above recent high + retest hold; MACD supportive.",
            ts_utc=str(df.index[-1]),
            confidence="B+"
        )

    # -------------------------
    # C) Failure SHORT (strict; optional)
    # Conditions:
    # - strong breakdown: last close < bb_mid and last close < ema20
    # - and retest rejection: prev close > ema20 but last close < ema20 with upper wick (high > ema20)
    # - trend_down or clear momentum shift
    # -------------------------
    rejection = (last["high"] > last["ema20"]) and (last["close"] < last["ema20"]) and (last["close"] < last["bb_mid"])
    momentum_shift = (last["macd_hist"] < prev["macd_hist"]) and (last["macd_hist"] < 0)

    if (trend_down or momentum_shift) and rejection:
        entry = float(last["close"])
        sl = float(last["high"] + (last["atr14"] * 0.25 if not np.isnan(last["atr14"]) else 0.5))
        r = sl - entry
        if r <= 0:
            return None
        tp1 = entry - 1.5 * r
        tp2 = entry - 2.5 * r
        return Signal(
            side="SHORT",
            strategy="Failure / Rejection",
            entry=_round(entry),
            sl=_round(sl),
            tp1=_round(tp1),
            tp2=_round(tp2),
            reason="Rejection at EMA20/BB-mid with bearish momentum; breakdown/retest failure.",
            ts_utc=str(df.index[-1]),
            confidence="B"
        )

    return None


# -----------------------------
# Tick -> candles
# -----------------------------

def read_ticks(conn: sqlite3.Connection, table: str, ts_col: str, price_col: str,
              since_ts: Optional[float]) -> pd.DataFrame:
    """
    Reads ticks newer than since_ts. Accepts ts as:
    - unix seconds (int/float)
    - unix ms (int > 1e12)
    - ISO datetime text (tries parse)
    """
    where = ""
    params = ()
    if since_ts is not None:
        # We'll filter in python too; SQLite filter only if numeric
        where = f""
    q = f"SELECT {ts_col} AS ts, {price_col} AS price FROM {table} ORDER BY {ts_col} ASC"
    df = pd.read_sql_query(q, conn)

    if df.empty:
        return df

    # Normalize timestamps
    ts = df["ts"]

    if np.issubdtype(ts.dtype, np.number):
        ts_num = ts.astype(float)
        # detect ms
        if ts_num.max() > 1e12:
            ts_num = ts_num / 1000.0
        df["dt"] = pd.to_datetime(ts_num, unit="s", utc=True)
        df["ts_s"] = ts_num
    else:
        df["dt"] = pd.to_datetime(ts, utc=True, errors="coerce")
        df = df.dropna(subset=["dt"])
        df["ts_s"] = df["dt"].astype("int64") / 1e9

    if since_ts is not None:
        df = df[df["ts_s"] > since_ts]

    df = df.sort_values("dt")
    df = df[["dt", "price", "ts_s"]]
    return df

def ticks_to_1m_candles(ticks: pd.DataFrame) -> pd.DataFrame:
    if ticks.empty:
        return pd.DataFrame()

    t = ticks.set_index("dt")["price"].astype(float)
    ohlc = t.resample("1min").ohlc()
    # We can add tick count as volume proxy
    vol = t.resample("1min").count().rename("volume")
    c = pd.concat([ohlc, vol], axis=1).dropna()
    return c


# -----------------------------
# Alerts: Console + Telegram
# -----------------------------

def send_telegram(text: str) -> bool:
    token = os.getenv("TELEGRAM_BOT_TOKEN", "").strip()
    chat_id = os.getenv("TELEGRAM_CHAT_ID", "").strip()
    if not token or not chat_id:
        return False

    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = json.dumps({"chat_id": chat_id, "text": text})
    req = urllib.request.Request(url, data=payload.encode("utf-8"), headers={"Content-Type": "application/json"})
    try:
        with urllib.request.urlopen(req, timeout=10) as resp:
            _ = resp.read()
        return True
    except Exception:
        return False

def format_signal(symbol: str, tf: str, sig: Signal) -> str:
    lines = []
    lines.append(f"[ALERT] {symbol} | tf={tf} | {sig.confidence} | {sig.strategy} | {sig.side}")
    lines.append(f"Entry: {sig.entry}")
    lines.append(f"SL:    {sig.sl}")
    lines.append(f"TP1:   {sig.tp1}")
    if sig.tp2 is not None:
        lines.append(f"TP2:   {sig.tp2}")
    lines.append(f"Time:  {sig.ts_utc}")
    lines.append(f"Why:   {sig.reason}")
    return "\n".join(lines)


# -----------------------------
# Main loop
# -----------------------------

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--db", default="marketdata.sqlite", help="SQLite database path")
    ap.add_argument("--symbol", default="XAUUSD", help="Symbol label used in alerts (cosmetic)")
    ap.add_argument("--tf", default="1m", help="Timeframe for alerts (cosmetic, fixed to 1m in logic)")
    ap.add_argument("--interval", type=int, default=60, help="Polling interval seconds")
    ap.add_argument("--table", default=None, help="Force table name")
    ap.add_argument("--ts_col", default=None, help="Force timestamp column name")
    ap.add_argument("--price_col", default=None, help="Force price column name")
    ap.add_argument("--cooldown_min", type=int, default=30, help="Min minutes between same-strategy alerts")
    ap.add_argument("--min_candles", type=int, default=150, help="Minimum candles before signals")
    args = ap.parse_args()

    if not os.path.exists(args.db):
        print(f"[monitor] ERROR: DB niet gevonden: {args.db}", file=sys.stderr)
        sys.exit(1)

    conn = sqlite3.connect(args.db, check_same_thread=False)

    try:
        table, ts_col, price_col = detect_tick_table(conn, args.table, args.ts_col, args.price_col)
    except Exception as e:
        print(f"[monitor] init ERROR: {e}", file=sys.stderr)
        sys.exit(2)

    print(f"[monitor] gestart | DB={args.db} | table={table} | ts={ts_col} | price={price_col} | symbol={args.symbol} | interval={args.interval}s")

    # State
    since_ts = None
    candle_cache = pd.DataFrame()
    last_alert: Dict[str, float] = {}  # key=strategy -> unix_ts

    while True:
        try:
            ticks = read_ticks(conn, table, ts_col, price_col, since_ts)
            if not ticks.empty:
                since_ts = float(ticks["ts_s"].max())

                new_candles = ticks_to_1m_candles(ticks)
                if not new_candles.empty:
                    # Merge into cache and keep last N candles
                    candle_cache = pd.concat([candle_cache, new_candles]).sort_index()
                    candle_cache = candle_cache[~candle_cache.index.duplicated(keep="last")]
                    candle_cache = candle_cache.tail(500)

                    if len(candle_cache) >= args.min_candles:
                        sig = compute_signals(candle_cache)
                        if sig:
                            key = f"{sig.strategy}:{sig.side}"
                            now = time.time()
                            last_time = last_alert.get(key, 0.0)
                            if now - last_time >= args.cooldown_min * 60:
                                msg = format_signal(args.symbol, args.tf, sig)
                                print(msg)
                                sent = send_telegram(msg)
                                if sent:
                                    print("[alert] sent to Telegram")
                                last_alert[key] = now

            time.sleep(args.interval)

        except KeyboardInterrupt:
            print("\n[monitor] gestopt (CTRL+C)")
            break
        except Exception as e:
            print(f"[monitor] ERROR: {e}", file=sys.stderr)
            time.sleep(max(5, args.interval))

    conn.close()


if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--db DB] [--symbol SYMBOL] [--tf TF]
                             [--interval INTERVAL] [--table TABLE]
                             [--ts_col TS_COL] [--price_col PRICE_COL]
                             [--cooldown_min COOLDOWN_MIN]
                             [--min_candles MIN_CANDLES]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/woutertimmer/Library/Jupyter/runtime/kernel-v3172eaf8845db01b13dac8c121b0665dbfda31cd9.json


SystemExit: 2

/Users/woutertimmer/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
